In [21]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

import tqdm as tqdm
from copy import deepcopy
import matplotlib.pyplot as plt

In [28]:
net = torchvision.models.mobilenet_v2(pretrained=False)
num_ftrs = net.last_channel

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

classifier = nn.Linear(num_ftrs, len(classes))

In [29]:
net_one = deepcopy(net)
net_one.cuda()
net_two = deepcopy(net)
net_two.cuda()
net_three = deepcopy(net)
net_three.cuda()
print('')

In [7]:
from crossngover import CrossN

cross = CrossN()

In [30]:
gen_one = next(iter(cross.breed([net_one,net_two,net_three])))
gen_two = next(iter(cross.breed([net_one,net_two,net_three])))


0it [00:00, ?it/s]

0it [00:00, ?it/s]


In [39]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [49]:
def validation(net, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        # try filter by loss too
        for data in tqdm(testloader):
                images, labels = data
                images = images.cuda()
                labels = labels.cuda()
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [50]:
def train_one_epoch(net, trainloader, testloader):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    loss = 0.0
    print('training: {}'.format(net.ancestry))
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    val_score = validation(net, testloader)
    print('[%d, %5d] loss: %.3f validation score: %.2f'.format
          (epoch + 1, i + 1, loss, val_score))
    return net

In [48]:
train_one_epoch(gen_one, trainloader, testloader)

In [33]:
def train_n_breed(nets, testloader):
    trained_nets = []
    for net in nets:
        net = train_one_epoch(net, trainloader, testlaoder)
        trained_nets.append(net)
    best_score = 0
    best_net = None
    for n in cross.breed(nets):
        score = validation(n, testloader)
        best_net = [None, None]
        if score > best_score:
            best_score = score
            best_net.pop(0)
            best_net.append(net)
    return best_nets, best_score

In [51]:
num_epochs = 5

for epoch in range(num_epochs):
    best_nets, best_score = train_n_breed(nets, testloader)
    nets = best_nets
    print('EPOCH:', epoch)
    print('Best_score {}'.format(best_score))
    for n in nets:
        cross.history(n)
        

NameError: name 'num_epochs' is not defined